In [1]:
import time
import pycurl
import urllib
import json
import requests
import numpy as np

In [2]:
API_ENDPOINT_URL = 'http://lauzhack.sqpub.ch/'
API_KEY = '98423jhHJDmM'

class SwissQuoteStream:
    def __init__(self):
        
        self.conn = None
        self.buffer = ''
        
        
    def setup_connection(self):
        if self.conn:
            self.conn.close()
            self.buffer = ''
        self.conn = pycurl.Curl()
        self.conn.setopt(pycurl.URL, API_ENDPOINT_URL + 'prices')
        self.conn.setopt(pycurl.WRITEFUNCTION, self.callback)
    
    def start(self, callback):
        self.callback = callback
        self.setup_connection()
        backoff_network_error = 0.25
        backoff_http_error = 5
        backoff_rate_limit = 60
        while True:
            self.setup_connection()
            try:
                self.conn.perform()
            except:
                print('Network error')
                time.sleep(backoff_network_error)
                backoff_network_error = min(backoff_network_error + 1, 16)
                continue
            # HTTP Error
            sc = self.conn.getinfo(pycurl.HTTP_CODE)

            # HTTP error, use exponential back off up to 320 seconds
           # print(HTTP error %s, %s' % (sc, self.conn.errstr()))
           # print 'Waiting %s seconds' % backoff_http_error
            time.sleep(backoff_http_error)
            backoff_http_error = min(backoff_http_error * 2, 320)
            
    def buy(self, btc_amount):
        #data = str.encode('BUY 0.00000001 BTC ' + API_KEY)
        data = str.encode('BUY ' + str(btc_amount) +' BTC ' + API_KEY)
        r = requests.post(API_ENDPOINT_URL, data=data)
        return str(r.status_code) + ' ' + str(r.reason)

        
    def sell(self, btc_amount):
        #data = str.encode('SELL 0.00000001 BTC ' + API_KEY)
        data = str.encode('SELL ' + str(btc_amount) +' BTC ' + API_KEY)
        r = requests.post(API_ENDPOINT_URL, data=data)
        return str(r.status_code) + ' ' + str(r.reason)
        


In [3]:
stream = SwissQuoteStream()

In [4]:
def extract_price(data):
    return float(data.decode("utf-8").split(' ')[1][:-1])

WINDOW_MAX_SIZE = 50

window = []
epsilon = 0.01
ignore_count = 0
ignore_n = 5

normalizer_max = 5

def diff_normalizer(diff):
    if(diff > normalizer_max):
        diff_tmp = normalizer_max
    elif(diff < -normalizer_max):
        diff_tmp = -normalizer_max
    else:
        diff_tmp = diff
        
    normalized = (diff_tmp + normalizer_max) / (normalizer_max * 2)
    
    return normalized

def handle_new_price(data):
    global ignore_count

    new_price = extract_price(data)
    window.append(new_price)
    window_resized = window[-WINDOW_MAX_SIZE:]
    window_size = len(window_resized)
    
    weights = np.exp(np.linspace(-1., 0., window_size))
    weights /= weights.sum()
    
    a = np.convolve(window_resized, weights, mode='full')[:len(window_resized)]
    ema = a[window_size -1]
    
    diff = new_price - ema
    # diff > 0 -> price is above ema -> SELL SELL SELL!

    ignore_count = (ignore_count + 1 ) % ignore_n
    
    if(ignore_count == 0):
        big_enough = abs(diff) > epsilon
        
        if(not big_enough):
            print('Just HODL. Diff is: ' + str(diff))
            return
        
        END = 'http://lauzhack.sqpub.ch/teams/'

        r = requests.get(END)
        assets = r.json()[12]["assets"]
        cash = r.json()[12]["cash"]
        total = assets + cash
        
        current_percent_btc = assets / total #percentage of current bitcoin compared to total balance
        normalized_diff = 1 - diff_normalizer(diff) #percentage of bitcoin we want to have, compared to total balance
        percent_btc_diff = normalized_diff - current_percent_btc #percentage of cash to add
        amount_of_btc = percent_btc_diff * total # amount of bitcoin we want to add
        amount_in_btc = amount_of_btc / new_price #amount of equivalent bitcoin we want to add in cash
        
        #print(str(total) + ' ' + str(assets) + ' ' + str(cash) + ' ||| ' + str(diff) + ' ' + str(current_percent_btc) + ' ' + str(percent_btc_diff) + ' ' + str(amount_of_btc) + ' ' + str(amount_in_btc))
        
        if(amount_in_btc < 0):
            aa = stream.sell(abs(amount_in_btc))
            print('Diff: ' + str(normalized_diff) + '. SELL ' + str(abs(amount_in_btc)))
        else:
            aa = stream.buy(abs(amount_in_btc))
            print('Diff: ' + str(normalized_diff) + '. BUY ' + str(abs(amount_in_btc)))


In [ ]:
stream.start(handle_new_price)

Diff: 1.0. BUY 11.128621783879794
Diff: 0.6370879056206377. BUY 1.546420749224869
Diff: 0.4840999512364761. SELL 4.027383001504515
Diff: 0.42495447787250673. SELL 1.556955773496804
Diff: 0.432519015207663. BUY 0.19906606984349567
Diff: 0.4408282646189946. BUY 0.22475702024740876
Diff: 0.19057898310707055. SELL 6.591924560136426
Diff: 0.3355689276836529. BUY 3.815857875247141
Diff: 0.5445905104993471. BUY 5.500127211790793
Diff: 0.23545369191815557. SELL 8.135514650085598
Diff: 0.1176702848488731. SELL 3.0987465313529707
Diff: 0.06671469106690897. SELL 1.3400662808813324
Diff: 0.12137138014873017. BUY 1.4367643583431076
Diff: 0.17271113387050718. BUY 1.3495424826439886
Diff: 0.225718240680726. BUY 1.393371931493623
Diff: 0.2823436514585864. BUY 1.4884441791044838
Diff: 0.333938078352503. BUY 1.3562075271377574
Diff: 0.3880014840346121. BUY 1.42108213848751
Diff: 0.4409659691321395. BUY 1.3919911078385414
Diff: 0.48004710707637055. BUY 1.024011745215578
Diff: 0.3057009440894035. SELL 4.5

Diff: 0.4413170525844634. BUY 1.440489561141962
Diff: 0.4133490658900427. SELL 0.7375402323487682
Diff: 0.4249805492474025. BUY 0.30672696502908253
Diff: 0.6951371609943635. BUY 7.131001965181172
Diff: 0.7549419059401771. BUY 1.5770996250847225
Diff: 0.6474779520254742. SELL 2.835996522837799
Diff: 0.6453691828398405. SELL 0.05524798260635037
Diff: 0.609318379170827. SELL 0.9509295435906516
Diff: 0.5870844663414573. SELL 0.5864049502579977
Diff: 1.0. BUY 10.905983419805954
Diff: 0.8492311208591218. BUY 6.9233929187212295
Diff: 0.8004417333026594. SELL 1.2863635452755537
Diff: 0.8802291781148597. BUY 2.1051992141374902
Diff: 0.9475966965429052. BUY 1.7783856683167307
Diff: 0.960273561796248. BUY 0.33521809609134584
Diff: 1.0. BUY 1.0484925557761238
Diff: 1.0. BUY 1.0495524832104122
Diff: 1.0. BUY 1.0495103817675264
Diff: 1.0. BUY 1.049207351059374
Diff: 1.0. BUY 2.1332655238412158e-05
Diff: 1.0. BUY 6.887754113309879e-09
Diff: 1.0. BUY 6.890408282009141e-09
Diff: 1.0. BUY 6.891479839795